In [2]:
import pandas as pd
from langchain.output_parsers import PandasDataFrameOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# Define your desired Pandas DataFrame.
#df=pd.read_excel("./pull_requests_summary.xlsx", engine='openpyxl')
df=pd.read_csv("./pull_requests_summary.csv")
df.head(20)


,Commit ID,Changes,Comments
0,1463a176bf652172560c3893c84b61c86e1b47f5,diff --git a/src/main/java/com/manh/cp/appoint...,NaN
1,39c8f648fdb08e8717237ae4ffca5b181cd3fd33,diff --git a/gradle.properties b/gradle.proper...,Question\(may be to BA\) : Can we have Day Lev...
2,949be2ac0e957cfbe981da242384424b4786067a,diff --git a/src/component-test/java/com/manh/...,why do we need to set this ?\r\nresolve sonar ...
3,4acc32ac9bf2dde41083bac73dc15f89cc69345d,diff --git a/build.gradle b/build.gradle\r\nin...,![](https://bitbucket.org/repo/9p65goR/images/...
4,97ac5a506b358d29c298aecadd41060d62cecc4b,diff --git a/src/component-test/java/com/manh/...,NaN
5,c1fcce1636d4e6a240fbaf6ffa6c30058c08e69e,diff --git a/src/component-test/java/com/manh/...,This change is made to correct the test case t...
6,3ecedef3e81ac4104fe3f0a028aafad496ed6b42,diff --git a/client/src/main/java/com/manh/cp/...,Replace 0 and 1 with the name of the item bein...
7,0fa74824f9fefec08acfe117866dc587563730d5,diff --git a/gradle.properties b/gradle.proper...,Lets not do a migration script. Just clean it ...
8,26d2d44b12c28580549b6ade52bb6a4949cfe521,diff --git a/src/component-test/java/com/manh/...,Do I also need to create a record in appointme...
9,8a493a8f89f9c9d108922bd83af39f776ee6c76b,diff --git a/gradle.properties b/gradle.proper...,can we add Assert.fail\(\) here?\r\nNo FF mock...


In [8]:
# Set up a parser + inject instructions into the prompt template.
parser = PandasDataFrameOutputParser(dataframe=df)

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) #gpt-3.5-turbo #gpt-4o-mini

instructions = parser.get_format_instructions()
print(instructions)

The output should be formatted as a string as the operation, followed by a colon, followed by the column or row to be queried on, followed by optional array parameters.
1. The column names are limited to the possible columns below.
2. Arrays must either be a comma-separated list of numbers formatted as [1,3,5], or it must be in range of numbers formatted as [0..4].
3. Remember that arrays are optional and not necessarily required.
4. If the column is not in the possible columns or the operation is not a valid Pandas DataFrame operation, return why it is invalid as a sentence starting with either "Invalid column" or "Invalid operation".

As an example, for the formats:
1. String "column:num_legs" is a well-formatted instance which gets the column num_legs, where num_legs is a possible column.
2. String "row:1" is a well-formatted instance which gets row 1.
3. String "column:num_legs[1,2]" is a well-formatted instance which gets the column num_legs for rows 1 and 2, where num_legs is a p

In [9]:
# Here's an example of a column operation being performed.
df_query = "how many rows in column:Changes"

# Set up the prompt.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": instructions},
)

chain = prompt | model | parser
response = chain.invoke({"query": df_query})
print(response)

OutputParserException: Invalid operation: "how many rows" is not a valid Pandas DataFrame operation.. Please check the format instructions.

In [1]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

In [2]:
import pandas as pd
from langchain_openai import OpenAI

df = pd.read_csv("./pull_requests_summary.csv")

In [4]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True, allow_dangerous_code=True)

In [9]:
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

In [10]:
agent.invoke("how many columns are there?")



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-3.5-turbo in organization org-W86eAuAi3cZi779N6hUw65Im on tokens per min (TPM): Limit 200000, Requested 774931. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}